In [2]:
import shutil
from glob import glob
import nrrd
import os
import PIL
from PIL import Image
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np
import matplotlib as plot
from matplotlib import pylab as plt
from preprocess.utils import *

In [ ]:
def normalize(img):
    Min=np.min(img)
    Max=np.max(img)
    img_new=(img-Min)/(Max-Min)
    return img_new

In [ ]:
'''
    获取x,y,z轴非0切面，即结节所在区域
    input：mask文件
    output：二维list，分别存放x,y,z轴的切面序号
'''
def getnonzeros(data:np.ndarray,Tran=False):
    if Tran:
        data=data.transpose((2,1,0))
    nonIndex=[[],[],[]]
    for i in range(0,3):
        for j in range(0,data.shape[i]):
            if i==0:
                temp=np.sum(data[j,:,:]).squeeze()
            elif i==1:
                temp=np.sum(data[:,j,:]).squeeze()
            elif i==2:
                temp=np.sum(data[:,:,j]).squeeze()
            if temp !=0:
                nonIndex[i].append(j)
    return nonIndex

In [2]:
#读取excel文件
data_path=r'F:\Download\1'
image_path=os.path.join(data_path,'image')
csv_path=os.path.join(data_path,'filtData.xlsx')
data=pd.read_excel(csv_path,usecols='A:E',dtype={0:'str',1:'int64',4:'int64'})
exfiles=['09652616']
print(data)

         file  type   x   y   z
0    00625858     0  14  16  17
1    01034481     0  11  10   8
2    02501737     1  14  19   8
3    03022694     1  14  13   4
4    03223027     0  17  16   5
..        ...   ...  ..  ..  ..
201  09823965     0  33  27  17
202  09835955     1  32  33  21
203  09836250     1  13  15  11
204  68103460     0  10  11   7
205  70063493     0  38  39  20

[206 rows x 5 columns]


In [3]:
"""
    以下x,y,z方向最大半径分割
    保存为图片
"""
#读取excel文件
data_path=r'F:\Download\1'
image_path=os.path.join(data_path,'image')
csv_path=os.path.join(data_path,'filtData.xlsx')
data=pd.read_excel(csv_path,usecols='A:E',dtype={0:'str',1:'int64',4:'int64'})
exfiles=['09652616']
print(data)
error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image5')
for i in range(len(data)):
    filename=str(data.iloc[i,0])[:-5]
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=(str(data.iloc[i,1]).strip())=='有'
    if type:
        type=1
    else:
        type=0

    image_file=os.path.join(image_path,filename)
    nrrd_data,_=nrrd.read(filepath)
    mask_data,_=nrrd.read(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.shape!=nrrd_data.shape:
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    nonIndex=getnonzeros(mask_data)
    #全局标准化
    # nrrd_data=zscore(nrrd_data)*255
    # nrrd_data=nrrd_data.astype(int)
    #ROI_data=nrrd_data[nonIndex[0][0]:nonIndex[0][-1]+1,nonIndex[1][0]:nonIndex[1][-1]+1,nonIndex[2][0]:nonIndex[2][-1]+1]
    #获取结节中心
    centor_w=int((nonIndex[0][0]+nonIndex[0][-1])/2)
    centor_h=int((nonIndex[1][0]+nonIndex[1][-1])/2)
    centor_n=int((nonIndex[2][0]+nonIndex[2][-1])/2)
    #获取结节x，y，z轴最大半径
    r=max(len(nonIndex[0]),len(nonIndex[1]),len(nonIndex[2]))
    r=int((r+1)/2)
    #分别在x,y,z轴三个方向切割，保存到0，1，2目录
    for i in range(0,3):
        target_file=os.path.join(image_path,filename,f'{i}')
        if not os.path.exists(target_file):
            os.makedirs(target_file)
        for j in nonIndex[i]:
            if i==0:
               img=nrrd_data[j,centor_h-r:centor_h+r,centor_n-r:centor_n+r]
            elif i==1:
                img=nrrd_data[centor_w-r:centor_w+r,j,centor_n-r:centor_n+r]
            elif i==2:
                img=nrrd_data[centor_w-r:centor_w+r,centor_h-r:centor_h+r,j]
            #局部标准化，图片看起来更加清晰
            img=zscore(img)*255
            img=img.astype(int)
            img=Image.fromarray(img)
            img.convert('L').save(os.path.join(target_file,f'{j}_{type}.png'))
    print(filename,':complete!')
print('错误文件：',error_file)


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\Download\\1\\006.nrrd'

In [5]:
'''
    基于固定半径分割
'''
#读取excel文件
data_path=r'F:\Download\1'
image_path=os.path.join(data_path,'image')
csv_path=os.path.join(data_path,'filtData.xlsx')
data=pd.read_excel(csv_path,usecols='A:E',dtype={0:'str',1:'int64',4:'int64'})
exfiles=['09652616']
print(data)
error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image7')
for i in range(len(data)):
    if data.iloc[i,4]>32:
        continue

    filename=str(data.iloc[i,0])
    #filename=str(data.iloc[i,0])[:-5]

    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    # type=(str(data.iloc[i,1]).strip())=='有'
    # if type:
    #     type=1
    # else:
    #     type=0
    type=data.iloc[i,1]
    image_file=os.path.join(image_path,filename)
    nrrd_data,_=nrrd.read(filepath)
    mask_data,_=nrrd.read(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.shape!=nrrd_data.shape:
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    nonIndex=getnonzeros(mask_data)
    #全局标准化
    # nrrd_data=zscore(nrrd_data)*255
    # nrrd_data=nrrd_data.astype(int)
    #nrrd_data=nrrd_data*mask_data
    #ROI_data=nrrd_data[nonIndex[0][0]:nonIndex[0][-1]+1,nonIndex[1][0]:nonIndex[1][-1]+1,nonIndex[2][0]:nonIndex[2][-1]+1]
    #获取结节中心
    centor_w=int((nonIndex[0][0]+nonIndex[0][-1])/2)
    centor_h=int((nonIndex[1][0]+nonIndex[1][-1])/2)
    centor_n=int((nonIndex[2][0]+nonIndex[2][-1])/2)
    #获取结节x，y，z轴最大半径
    r=max(len(nonIndex[0]),len(nonIndex[1]),len(nonIndex[2]))
    if r>32:
        continue
    r=16
    #分别在x,y,z轴三个方向切割，保存到0，1，2目录
    for i in range(0,3):
        target_file=os.path.join(image_path,filename,f'{i}')
        if not os.path.exists(target_file):
            os.makedirs(target_file)
        for j in nonIndex[i]:
            if i==0:
               img=nrrd_data[j,centor_h-r:centor_h+r,centor_n-r:centor_n+r]
            elif i==1:
                img=nrrd_data[centor_w-r:centor_w+r,j,centor_n-r:centor_n+r]
            elif i==2:
                img=nrrd_data[centor_w-r:centor_w+r,centor_h-r:centor_h+r,j]
            #局部标准化，图片看起来更加清晰
            img=zscore(img)*255
            img=img.astype(int)
            img=Image.fromarray(img)
            img.convert('L').save(os.path.join(target_file,f'{j}_{type}.png'))
    print(filename,':complete!')
print('错误文件：',error_file)



00625858 :complete!
01034481 :complete!
02501737 :complete!
03022694 :complete!
03223027 :complete!
03571004 :complete!
04183261 :complete!
04247827 :complete!
04595852 :complete!
04776614 :complete!
04873120 :complete!
05256788 :complete!
05376764 :complete!
05412252 :complete!
05431443 :complete!
05545138 :complete!
05587338 :complete!
05688162 :complete!
05738378 :complete!
05807742 :complete!
05849467 :complete!
05871891 :complete!
05930651 :complete!
05968948 :complete!
06025976 :complete!
06135499 :complete!
06260590 :complete!
07136826 :complete!
07156597 :complete!
07156637 :complete!
07157615 :complete!
07238217 :complete!
07280060 :complete!
07289679 :complete!
07459320 :complete!
07592297 :complete!
07723928 :complete!
07870634 :complete!
07942262 :complete!
08031582 :complete!
08127850 :complete!
08291767 :complete!
08350481 :complete!
08416480 :complete!
08516985 :complete!
08555483 :complete!
08603817 :complete!
08651569 :complete!
08734184 :complete!
08824935 :complete!


In [ ]:
''':cvar
    尝试添加 临床数据，保存为numpy 数组.npy文件
'''
#读取excel文件
data_path=r'F:\Download\1'
image_path=os.path.join(data_path,'image')
csv_path=os.path.join(data_path,'filtData.xlsx')
data=pd.read_excel(csv_path,usecols='A:E',dtype={0:'str',1:'int64',4:'int64'})
exfiles=['09652616']
print(data)

data_path=r'F:\Download\1'
image_path=os.path.join(data_path,'image')
csv_path=os.path.join(data_path,'results_20220822.xlsx')
data=pd.read_excel(csv_path,usecols='B:I')
#去除空值
data=data.dropna()
#去除 空格等符号
data.replace('\s+','',regex=True,inplace=True)
print(data.shape)

#
label=pd.concat((data.iloc[:,0],data.iloc[:,-1]),axis=1)
clidata=data.iloc[:,1:-2]
print(label.shape)
#离散数据one-hot处理
clidata=pd.get_dummies(clidata,columns=['性别','有无吸烟'])
#年龄按四分位数分层处理
bins=[0,46,53,58.5,100]
clidata['年龄']=pd.cut(clidata['年龄'],bins,labels=range(1,5))
print(clidata)
print(clidata.columns.values)
print(clidata.shape)



In [ ]:
"""
    以下x,y,z方向最大半径分割
    结合临床数据保存为numpy数组，[0]为切片数据,[1]为临床数据
"""
#读取excel文件
data_path=r'F:\Download\1'
image_path=os.path.join(data_path,'image')
csv_path=os.path.join(data_path,'filtData.xlsx')
data=pd.read_excel(csv_path,usecols='A:E',dtype={0:'str',1:'int64',4:'int64'})
exfiles=['09652616']
print(data)

exfiles=['09652616']
error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image_cli')
for i in range(len(label)):
    filename=str(label.iloc[i,0])[:-5]
    cdata=np.array(clidata.iloc[i,:])
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=(str(label.iloc[i,1]).strip())=='有'
    if type:
        type=1
    else:
        type=0

    image_file=os.path.join(image_path,filename)
    nrrd_data,_=nrrd.read(filepath)
    mask_data,_=nrrd.read(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.shape!=nrrd_data.shape:
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    nonIndex=getnonzeros(mask_data)
    #全局标准化
    # nrrd_data=zscore(nrrd_data)*255
    # nrrd_data=nrrd_data.astype(int)
    #ROI_data=nrrd_data[nonIndex[0][0]:nonIndex[0][-1]+1,nonIndex[1][0]:nonIndex[1][-1]+1,nonIndex[2][0]:nonIndex[2][-1]+1]
    #获取结节中心
    centor_w=int((nonIndex[0][0]+nonIndex[0][-1])/2)
    centor_h=int((nonIndex[1][0]+nonIndex[1][-1])/2)
    centor_n=int((nonIndex[2][0]+nonIndex[2][-1])/2)
    #获取结节x，y，z轴最大半径
    r=max(len(nonIndex[0]),len(nonIndex[1]),len(nonIndex[2]))
    r=int((r+1)/2)
    #分别在x,y,z轴三个方向切割，保存到0，1，2目录
    for i in range(0,3):
        target_file=os.path.join(image_path,filename,f'{i}')
        if not os.path.exists(target_file):
            os.makedirs(target_file)
        for j in nonIndex[i]:
            if i==0:
               img=nrrd_data[j,centor_h-r:centor_h+r,centor_n-r:centor_n+r]
            elif i==1:
                img=nrrd_data[centor_w-r:centor_w+r,j,centor_n-r:centor_n+r]
            elif i==2:
                img=nrrd_data[centor_w-r:centor_w+r,centor_h-r:centor_h+r,j]
            #局部标准化，图片看起来更加清晰
            img=zscore(img)*255
            img=img.astype(int)
            np.savez(os.path.join(target_file,f'{j}_{type}'),image=img,cdata=cdata)
            #img=Image.fromarray(img)
            #img.convert('L').save(os.path.join(target_file,f'{j}_{type}.png'))
    print(filename,':complete!')
print('错误文件：',error_file)

In [3]:
import SimpleITK as sitk

"""
    以下x,y,z方向最大半径分割
    结合临床数据保存为numpy数组，[0]为切片数据,[1]为临床数据
"""
#读取excel文件
OUTSIZE=[32,32,32]
data_path=r'F:\Download\1'
csv_path=os.path.join(data_path,'results_20220822.xlsx')
data=pd.read_excel(csv_path,usecols='B:I')
exfiles=['09652616']
print(data)

          fileName  结节性质（纯=0，混合=1）  结节大小（≤1cm=0，＞1cm=1） 性别  年龄 有无吸烟  \
0    00625858.nrrd             0.0                  1.0  女  61    无   
1    01034481.nrrd             1.0                  0.0  女  47    无   
2    02501737.nrrd             1.0                  1.0  男  41    有   
3    03022694.nrrd             0.0                  0.0  男  47    有   
4    03223027.nrrd             0.0                  1.0  女  63    无   
..             ...             ...                  ... ..  ..  ...   
203  09823965.nrrd             0.0                  1.0  男  59    有   
204  09835955.nrrd             1.0                  1.0  女  49    无   
205  09836250.nrrd             1.0                  1.0  女  41    无   
206  68103460.nrrd             0.0                  0.0  男  44   有    
207  70063493.nrrd             1.0                  1.0  男  44    有   

          病理类型 EGFR突变情况  
0      右上肺原位腺癌        无  
1    右上肺微浸润性腺癌        无  
2     右上肺浸润性腺癌        有  
3     左肺微浸润性腺癌        有  
4    左上肺微浸润性腺癌   

In [4]:
error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image1')
for i in range(len(data)):
    filename=str(data.iloc[i,0])[:-5]
    cdata=np.array(data.iloc[i,:])
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=int(data.iloc[i,1])

    image_file=os.path.join(image_path,filename)
    origin_data=sitk.ReadImage(filepath)
    mask_data=sitk.ReadImage(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.GetSize()!=origin_data.GetSize():
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    #注意sitk.GetArrayFromImage将图片又[x,y,z]变为[z,x,y]
    nonIndex=getnonzeros(sitk.GetArrayFromImage(mask_data))
    #设置窗口并缩放到[0,255]
    #origin_data=setWindow(origin_data)
    #连通域分割
    mask_data=sitk.ConnectedComponent(mask_data)
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(mask_data)
    #分割最大连通域
    area_max_label=0
    area_max=0
    for i in range(1, label_shape_filter.GetNumberOfLabels()+1):
        area=label_shape_filter.GetNumberOfPixels(i)
        if area>area_max:
            area_max_label = i
            area_max = area
    centor=origin_data.TransformPhysicalPointToIndex(label_shape_filter.GetCentroid(area_max_label))
    x,y,z,w,h,d=label_shape_filter.GetBoundingBox(area_max_label)
    r=int(max(w,h,d)*1.2)
    start_x,start_y,start_z=int(x-(r-w)/2),int(y-(r-h)/2),int(z-(r-d)/2)
    ROI_data=sitk.RegionOfInterest(origin_data,index=[start_x,start_y,start_z],size=[r,r,r])

    ROI_data=resampleSize(ROI_data,OUTSIZE)
    ndata=sitk.GetArrayFromImage(ROI_data)
    target_path=os.path.join(image_path,f'{type}')
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    np.savez(os.path.join(target_path,filename+'.npz'),image=ndata,cdata=np.zeros(shape=(5,)))
    #img=Image.fromarray(img)
    #img.convert('L').save(os.path.join(target_file,f'{j}_{type}.png'))
    print(filename,':complete!')
print('错误文件：',error_file)


00625858 :complete!
01034481 :complete!
02501737 :complete!
03022694 :complete!
03223027 :complete!
03571004 :complete!
04183261 :complete!
04247827 :complete!
04595852 :complete!
04693158 :complete!
04776614 :complete!
04873120 :complete!
05118583 :complete!
05256788 :complete!
05376764 :complete!
05412252 :complete!
05431443 :complete!
05545138 :complete!
05587338 :complete!
05688162 :complete!
05738378 :complete!
05807742 :complete!
05849467 :complete!
05871891 :complete!
05930651 :complete!
05968948 :complete!
06025976 :complete!
06135499 :complete!
06137339 :complete!
06260590 :complete!
07056211 :complete!
07136826 :complete!
07156597 :complete!
07156637 :complete!
07157615 :complete!
07238217 :complete!
07280060 :complete!
07289679 :complete!
07459320 :complete!
07592297 :complete!
07723928 :complete!
07870634 :complete!
07942262 :complete!
08031582 :complete!
08127850 :complete!
08291767 :complete!
08350481 :complete!
08416480 :complete!
08516985 :complete!
08555483 :complete!
